In [2]:
#!pip install -r requirements.txt

In [7]:
from elasticsearch import Elasticsearch

# Verbindung herstellen

In [9]:
with open('secret.txt','r') as f:
    connection=f.read()
print(connection)

http://10.0.0.168:9200


In [10]:
es = Elasticsearch([connection])
es.info()

ObjectApiResponse({'name': '5023a93705cc', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'bU3iWkjPQ_u0F4VrBeHiXQ', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
# Check Connection
if es.ping():
    print("Elasticsearch is connected!")
else:
    print("Elasticsearch is not connected!")

Elasticsearch is connected!


In [ ]:
# Überprüfen, ob Elasticsearch erreichbar ist
def check_connection():
    if es.ping():
        print("Elasticsearch is connected!")
    else:
        print("Elasticsearch is not connected!")

# Index erstellen
def create_index(index_name):
    es.indices.create(index=index_name)

# Index löschen
def delete_index(index_name):
    es.indices.delete(index=index_name)

# Dokument indizieren
def index_document(index_name, doc_id, document):
    es.index(index=index_name, id=doc_id, body=document)

# Dokument abrufen
def get_document(index_name, doc_id):
    return es.get(index=index_name, id=doc_id)

# Dokument aktualisieren
def update_document(index_name, doc_id, update_body):
    es.update(index=index_name, id=doc_id, body=update_body)

# Dokument löschen
def delete_document(index_name, doc_id):
    es.delete(index=index_name, id=doc_id)

# Einfache Suche
def search(index_name, query):
    return es.search(index=index_name, body={"query": {"match": query}})

# Beispielverwendung
if __name__ == "__main__":
    check_connection()
    index_name = "example_index"
    doc_id = 1
    document = {"name": "John Doe", "age": 30, "city": "New York"}

    create_index(index_name)
    index_document(index_name, doc_id, document)
    print("Indexed document:", get_document(index_name, doc_id))

    update_body = {"doc": {"age": 31}}
    update_document(index_name, doc_id, update_body)
    print("Updated document:", get_document(index_name, doc_id))

    query = {"name": "John Doe"}
    print("Search results:", search(index_name, query))

    delete_document(index_name, doc_id)
    delete_index(index_name)